In [1]:
import pickle as pk
import dgl as d
import pandas as pd
import numpy as np
from datasets.dataset import FeatureLookup

In [2]:
class a:
    def __init__(self, node2vec_dim, node2vec_length, node2vec_walk, node2vec_epochs, node2vec_batchsize):
        self.node2vec_dim = node2vec_dim
        self.node2vec_length = node2vec_length
        self.node2vec_walk = node2vec_walk
        self.node2vec_epochs = node2vec_epochs
        self.node2vec_batchsize = node2vec_batchsize
        self.node2vec_windows = 3
        self.slices = 10

# node2vec
def eraser(datasets, args):
    label = None

    # try:
    # label = pk.load(open(f'./pretrain/user_based_{args.slices}.pk', 'rb'))
    # except IOError:

    userg = d.graph([])
    user_lookup = FeatureLookup()
    ufile = pd.read_csv('./datasets/data/WSDREAM/原始数据/userlist_table.csv')
    ufile = pd.DataFrame(ufile)
    ulines = ufile.to_numpy()

    for i in range(339):
        user_lookup.register('User', i)

    for ure in ulines[:, 2]:
        user_lookup.register('URE', ure)

    for uas in ulines[:, 4]:
        user_lookup.register('UAS', uas)

    userg.add_nodes(len(user_lookup))

    for line in ulines:
        uid = line[0]
        ure = user_lookup.query_id(line[2])

        if not userg.has_edges_between(uid, ure):
            userg.add_edges(uid, ure)

        uas = user_lookup.query_id(line[4])
        if not userg.has_edges_between(uid, uas):
            userg.add_edges(uid, uas)

    userg = d.add_self_loop(userg)
    userg = d.to_bidirected(userg)

    from node2vec import Node2Vec
    G = userg.to_networkx()

    # 设置node2vec参数
    node2vec = Node2Vec(
        G,
        dimensions=args.node2vec_dim,  # 嵌入维度
        p=1,  # 回家参数
        q=0.5,  # 外出参数
        walk_length=args.node2vec_length,  # 随机游走最大长度
        num_walks=args.node2vec_walk,  # 每个节点作为起始节点生成的随机游走个数
        workers=4  # 并行线程数
    )

    # p=1, q=0.5, n_clusters=6。DFS深度优先搜索，挖掘同质社群
    # p=1, q=2, n_clusters=3。BFS宽度优先搜索，挖掘节点的结构功能。

    # 训练Node2Vec，参数文档见 gensim.models.Word2Vec
    model = node2vec.fit(
        window=args.node2vec_windows,  # Skip-Gram窗口大小
        epochs=args.node2vec_epochs,
        min_count=1,  # 忽略出现次数低于此阈值的节点（词）
        batch_words=args.node2vec_batchsize  # 每个线程处理的数据量
    )
    ans = model.wv.vectors[:339]

    df = np.array(ans)

    split_train_Tensor = []
    # 整理特征
    pk.dump(df, open(f'./pretrain/user_embeds.pk', 'wb'))
    # return df

    def k_mean(inputs, k):
        from sklearn.cluster import KMeans
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(inputs)
        return kmeans.labels_

    label = k_mean(df, args.slices)
    pk.dump(label, open(f'./pretrain/user_based_{args.slices}.pk', 'wb'))

    # 检查每个类的个数
    dic = {}
    for i in label:
        if i not in dic:
            dic[i] = 1
        else:
            dic[i] = dic[i] + 1
    print(dic)

    # # 获得切分矩阵
    # for i in range(args.slices):
    #     split_tensor = np.zeros_like(datasets)
    #     Idx = (label == i).nonzero()[0]
    #     for idx in Idx:
    #         split_tensor[idx] = datasets[idx]
    #     split_train_Tensor.append(split_tensor)

    return label

# 这里是做node2vec获得嵌入向量，然后通过嵌入向量获得各个类的集合

In [3]:
args = a(128, 20, 50, 20, 32)
label = eraser(None, args)

Computing transition probabilities:   0%|          | 0/507 [00:00<?, ?it/s]

{5: 110, 7: 49, 9: 15, 6: 17, 3: 27, 4: 23, 8: 10, 2: 19, 0: 19, 1: 50}


# 简单展示各个类中都有谁

In [4]:
C = [[] for _ in range(args.slices)]
for i in range(args.slices):
    for id, j in enumerate(label):
        if j == i:
            C[i].append(id)
C = np.array(C, dtype = 'object')
C

array([list([21, 25, 36, 39, 43, 61, 64, 65, 78, 81, 84, 96, 104, 124, 127, 131, 141, 253, 267]),
       list([30, 56, 58, 75, 90, 109, 135, 144, 145, 149, 151, 156, 159, 161, 168, 170, 175, 176, 177, 179, 184, 199, 214, 215, 216, 221, 224, 228, 231, 235, 237, 243, 244, 249, 250, 251, 252, 259, 263, 269, 280, 285, 289, 290, 295, 302, 311, 322, 326, 334]),
       list([15, 29, 38, 41, 44, 45, 97, 101, 102, 121, 132, 138, 147, 160, 198, 217, 279, 328, 333]),
       list([6, 8, 9, 10, 11, 18, 46, 50, 53, 62, 63, 66, 67, 74, 79, 80, 85, 158, 183, 189, 222, 223, 262, 296, 319, 331, 335]),
       list([12, 37, 40, 42, 47, 69, 71, 88, 98, 103, 111, 115, 119, 120, 126, 137, 140, 142, 155, 162, 165, 167, 173]),
       list([0, 16, 23, 24, 26, 31, 33, 51, 55, 57, 59, 60, 72, 82, 94, 125, 133, 134, 148, 152, 163, 164, 169, 172, 174, 182, 185, 186, 187, 188, 192, 193, 194, 195, 196, 197, 200, 201, 203, 204, 206, 207, 208, 210, 211, 212, 213, 218, 219, 225, 226, 227, 229, 230, 232, 233, 236, 239, 2

# 正式执行区域

In [5]:
def E_score2(value1, value2):
    return np.sum(np.power(value1 - value2, 2))

# 这个是嵌入向量
pretrain = pk.load(open(f'./pretrain/user_embeds.pk', 'rb'))

# 先获得每一个类的index，对应的类的数量，对应类的全部迁入特征的均值，已经类中的用户节点
center_mean_value = []
center_ = []
for i in range(args.slices):
    temp = []
    for j in C[i]:
        temp.append(pretrain[j])
    center_mean_value.append(np.mean(temp, axis = 0).tolist())
    # print(center_mean_value[i])
    center_.append([i, len(C[i]), center_mean_value[i], C[i]])

center_ = np.array(sorted(center_, key = (lambda x : x[1])), dtype = 'object')

center_

array([[8, 10,
        list([0.012842552736401558, 0.31468936800956726, 0.1255570948123932, 0.8877729177474976, 0.27440568804740906, -0.1766984462738037, 0.5243438482284546, -0.01825239323079586, 0.0570579469203949, 0.0944271832704544, 0.0376087911427021, 0.28073233366012573, 0.14996205270290375, -0.14195780456066132, -0.424160897731781, 0.4413203299045563, -0.3101520836353302, -0.3024551272392273, -0.2286977469921112, 0.4122544229030609, 0.050754498690366745, 0.34347060322761536, -0.4028100073337555, -0.41719236969947815, -0.7267547249794006, 0.4799768328666687, -0.08444611728191376, 0.31378501653671265, 0.12750951945781708, 0.1059451699256897, -0.06486334651708603, 0.19745513796806335, -0.3980278968811035, 0.08663026988506317, -0.5451751947402954, 0.6840831637382507, 0.36449089646339417, -0.3528314232826233, 0.4216586649417877, 0.7233584523200989, 0.007223218679428101, 0.49392399191856384, 0.31224027276039124, 0.5148701071739197, 0.5473910570144653, 0.3308292627334595, -0.16365256905

In [6]:
# 因为 1类 和 9类合并，我重新定义他的标号为 1类（最小类），设置是否访问就是为了后续不再加入9类
vis = [False for _ in range(args.slices)]

set_changed = True

# 开始执行
cnt = 0
while set_changed:
    set_changed = False

    # 遍历全部类
    for i in range(len(center_)):
        # 如果这个类的数量大于40，就代表他已经不需要合并了，或者他已经被合并过了，跳过
        if center_[i][1] >= 40 or vis[i]:
            continue

        # 寻找距离当前类i 最近的 类goal
        min_dis = 1e9
        goal = -1
        for j in range(i + 1, len(center_)):
            if vis[j]:
                continue
            # 如果这两个最近的类合并数量大于阈值，跳过
            if center_[i][1] + center_[j][1] <= 40:
                # 如果这个类最近，就是他了
                now_dis_between_i_and_j = E_score2(np.array(center_[i][2]), np.array(center_[j][2]))
                if now_dis_between_i_and_j < min_dis:
                    min_dis = now_dis_between_i_and_j
                    goal = j
        break

    # 存储合并的类a,和类b
    a = i
    b = goal

    # 获得这两个类的全部信息
    a_ = center_[a]
    b_ = center_[b]

    # 如果确定要合并，goal不是-1
    if goal != -1:
        print(f'{a_[0]} 有 {a_[1]} 个')
        print(f'{b_[0]} 有 {b_[1]} 个')
        print(a_[0], b_[0], '合并后 : ', a_[1] + b_[1], '个')
        # 让后续加进所有类的时候不重复加，所以已访问
        vis[a_[0]] = True
        vis[b_[0]] = True
        set_changed = True
        cnt += 1

        # 打印合并之前各个类的数量
        print('-' * 10)
        for i in center_:
            print(i[0], i[1])

    # 合并这两个集合
    merge_set = a_[3] + b_[3]
    center_mean_value = []
    # 获得这两个集合中的全部的特征向量并取均值
    for i in range(len(merge_set)):
        center_mean_value.append(pretrain[i])
    mean_value_a_and_b = np.mean(center_mean_value, axis = 0).tolist()

    # 这个就是合并的类，标号取两个类的最小，合并数量相加，合并之后，再取这两个类的全部特征取平均作为这个类的均值，以及合并之后集合里面都有哪些用户
    c_ = [min(a_[0], b_[0]), a_[1] + b_[1], mean_value_a_and_b, merge_set]

    # 将合并的集合与剩下的集合整理到一起

    # center_next是更新之后的总样本
    center_next = []
    center_mean_value = []
    for i in range(args.slices):
        temp = []

        # 如果已经合并了，就不需要重复加这两个原来的类
        if vis[i]:
            center_mean_value.append(0)
            continue

        # 就把其他没有合并的类加进来，并且每一个类依然是取均值作为这个类的特征
        for j in C[i]:
            temp.append(pretrain[j])
        center_mean_value.append(np.mean(temp, axis = 0).tolist())
        center_next.append([i, len(C[i]), center_mean_value[i], C[i]])

    # 因为我是以最小号 1，9 。取1类为这两个合并之后的标号，所以这个类取消访问
    vis[min(a_[0], b_[0])] = False
    # 并且重置1类中的集合为这个合并的集合
    C[min(a_[0], b_[0])] = merge_set

    # print(c_[0], c_[1], c_[3])

    # 将合并之后的集合加入到新的总样本中
    if goal != -1:
        center_next.append(c_)
        print('-' * 10)

    # 打印合并之前的全部类的index和数量
    if goal != -1:
        for i in center_next:
             print(i[0], i[1])


    # 这个排序无关紧要，就是方便理解，按类的数量上下排序
    center_next = np.array(sorted(center_next, key = (lambda x : x[1])), dtype = 'object')

    # 更新总样本
    center_ = center_next
    print('-' * 80)

    # break
print(f'合并了{cnt}次')

8 有 10 个
3 有 27 个
8 3 合并后 :  37 个
----------
8 10
9 15
6 17
0 19
2 19
4 23
3 27
7 49
1 50
5 110
----------
0 19
1 50
2 19
4 23
5 110
6 17
7 49
9 15
3 37
--------------------------------------------------------------------------------
9 有 15 个
0 有 19 个
9 0 合并后 :  34 个
----------
9 15
6 17
0 19
2 19
4 23
3 37
7 49
1 50
5 110
----------
1 50
2 19
3 37
4 23
5 110
6 17
7 49
0 34
--------------------------------------------------------------------------------
6 有 17 个
4 有 23 个
6 4 合并后 :  40 个
----------
6 17
2 19
4 23
0 34
3 37
7 49
1 50
5 110
----------
0 34
1 50
2 19
3 37
5 110
7 49
4 40
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
合并了3次


# 合并之后的全部样本

In [7]:
center_

array([[2, 19,
        list([0.1229032352566719, -0.2318892776966095, 0.04443539306521416, 0.581488311290741, 0.009204365313053131, 0.0647863820195198, 0.1649882197380066, -0.10589852929115295, -0.09644994884729385, 0.7240985035896301, -0.057512570172548294, 0.23262038826942444, 0.21698248386383057, -0.2807075083255768, -0.2468537539243698, 0.7063703536987305, -0.28074488043785095, -0.09544523060321808, -0.4764479100704193, 0.6294748187065125, 0.22055096924304962, 0.154939204454422, -0.18519611656665802, -0.4365135431289673, -0.710348904132843, 0.3206655979156494, -0.19056646525859833, 0.029124611988663673, 0.06087908148765564, 0.23122267425060272, -0.05759258568286896, 0.20632299780845642, -0.4184670150279999, 0.12194839119911194, -0.19903194904327393, 0.09342148900032043, 0.22339875996112823, 0.13239197432994843, 0.1434154063463211, 0.07182583957910538, -0.2881706953048706, 0.1374748796224594, -0.14848819375038147, 0.2993636727333069, 0.506003201007843, 0.21953420341014862, -0.407372

In [8]:
len(center_)

7

# 这一步是我做的第一个尝试，对富余的桶抽出来想再简单利用kmeans做聚类，特征采用的还是图嵌入，但是类的数量基本没有变化

In [9]:
def k_mean(inputs, k):
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(inputs)
    return kmeans.labels_

In [10]:
# 将所有超过这个桶的阈值数量的类抽取出来
extra_set = []
passed = 0
for item in center_:
    passed += 1
    if item[1] >= 40:
        extra_set += item[3]
np.array(extra_set)

array([  4,  54,  95, 105, 108, 116, 129, 130, 150, 157, 166, 181, 191,
       205, 240, 247, 258,  12,  37,  40,  42,  47,  69,  71,  88,  98,
       103, 111, 115, 119, 120, 126, 137, 140, 142, 155, 162, 165, 167,
       173,   1,   2,   5,  13,  17,  19,  20,  22,  27,  28,  32,  34,
        35,  48,  49,  52,  77,  83,  86,  87,  89,  91,  93, 112, 113,
       114, 117, 118, 122, 128, 136, 139, 146, 154, 178, 180, 190, 202,
       220, 234, 260, 264, 270, 274, 299, 320, 321, 332, 337,  30,  56,
        58,  75,  90, 109, 135, 144, 145, 149, 151, 156, 159, 161, 168,
       170, 175, 176, 177, 179, 184, 199, 214, 215, 216, 221, 224, 228,
       231, 235, 237, 243, 244, 249, 250, 251, 252, 259, 263, 269, 280,
       285, 289, 290, 295, 302, 311, 322, 326, 334,   0,  16,  23,  24,
        26,  31,  33,  51,  55,  57,  59,  60,  72,  82,  94, 125, 133,
       134, 148, 152, 163, 164, 169, 172, 174, 182, 185, 186, 187, 188,
       192, 193, 194, 195, 196, 197, 200, 201, 203, 204, 206, 20

In [11]:
# 整理信息，获得的是每个用户及其嵌入向量
now_set = []
for i in extra_set:
    now_set.append([i, pretrain[i].tolist()])
now_set = np.array(now_set, dtype = 'object')
inputs = np.array(now_set[:,1].tolist())
now_set

array([[4,
        list([-0.2114294469356537, -0.7388371229171753, -0.02856622450053692, 0.2367936372756958, 0.10003043711185455, -0.3259037435054779, 0.17803382873535156, 0.08528465777635574, -0.5376970171928406, 0.2694755792617798, 0.18787387013435364, 0.9624128341674805, -0.05300372838973999, -0.32376307249069214, 0.004467788152396679, 0.19517964124679565, -0.4569323658943176, 0.3982137441635132, -0.41772639751434326, 0.21047906577587128, -0.2351284772157669, -0.0035452309530228376, -0.21868370473384857, -0.07788693159818649, -0.45642390847206116, 0.39363619685173035, 0.10047077387571335, -0.04223979264497757, 0.28338780999183655, -0.19137190282344818, -0.24800191819667816, 0.19586513936519623, -0.2125115692615509, 0.16866032779216766, 0.012626193463802338, 0.29284220933914185, 0.05347434803843498, 0.053378161042928696, 0.09477818757295609, -0.02657831646502018, -0.016221212223172188, 0.563371479511261, -0.45146921277046204, 0.4324885904788971, 0.4318751096725464, -0.134654551744461

In [12]:
# 做第一个想法的kmeans聚类，结果如下所示
# 除去已经确定的桶，让美国大类再做聚类，放到剩下的十个桶里面 所以是  k = 10 - 已经确定好的桶数量
label = k_mean(inputs, args.slices - passed)
dic = {}
for i in range(len(label)):
    if label[i] not in dic:
        dic[label[i]] = 1
    else:
        dic[label[i]] += 1
dic

{2: 54, 1: 35, 0: 160}

# 开启第二个想法，利用上下文的国家和地域做聚类

In [13]:
# 和上面一样，整理多出来的用户，美国大类居多
extra_set = []
passed = 0
for item in center_:
    if item[1] >= 40:
        extra_set += item[3]
    else:
        passed += 1
np.array(extra_set)

array([  4,  54,  95, 105, 108, 116, 129, 130, 150, 157, 166, 181, 191,
       205, 240, 247, 258,  12,  37,  40,  42,  47,  69,  71,  88,  98,
       103, 111, 115, 119, 120, 126, 137, 140, 142, 155, 162, 165, 167,
       173,   1,   2,   5,  13,  17,  19,  20,  22,  27,  28,  32,  34,
        35,  48,  49,  52,  77,  83,  86,  87,  89,  91,  93, 112, 113,
       114, 117, 118, 122, 128, 136, 139, 146, 154, 178, 180, 190, 202,
       220, 234, 260, 264, 270, 274, 299, 320, 321, 332, 337,  30,  56,
        58,  75,  90, 109, 135, 144, 145, 149, 151, 156, 159, 161, 168,
       170, 175, 176, 177, 179, 184, 199, 214, 215, 216, 221, 224, 228,
       231, 235, 237, 243, 244, 249, 250, 251, 252, 259, 263, 269, 280,
       285, 289, 290, 295, 302, 311, 322, 326, 334,   0,  16,  23,  24,
        26,  31,  33,  51,  55,  57,  59,  60,  72,  82,  94, 125, 133,
       134, 148, 152, 163, 164, 169, 172, 174, 182, 185, 186, 187, 188,
       192, 193, 194, 195, 196, 197, 200, 201, 203, 204, 206, 20

# 国家和地域

In [14]:
ufile = pd.read_csv('./datasets/data/WSDREAM/原始数据/userlist_idx.csv').to_numpy()
inputs = []
for userid in extra_set:
    inputs.append([ufile[userid][2], ufile[userid][4]])
inputs = np.array(inputs)
label = k_mean(inputs, args.slices - passed)

dic = {}
for i in range(len(label)):
    if label[i] not in dic:
        dic[label[i]] = 1
    else:
        dic[label[i]] += 1
dic

{5: 36, 0: 38, 2: 38, 6: 34, 3: 42, 1: 33, 4: 28}

# 从结果上看，基本上都是满足了我的阈值

In [15]:
# 这个时候将各个类放回到对应的集合，方便后续放到桶里面
extra_set = np.array(extra_set)
print(extra_set.shape, label.shape)
extra_set = np.concatenate([extra_set.reshape(-1, 1), label.reshape(-1, 1)], axis = 1)
extra_set

(249,) (249,)


array([[  4,   5],
       [ 54,   0],
       [ 95,   2],
       [105,   6],
       [108,   0],
       [116,   3],
       [129,   6],
       [130,   3],
       [150,   2],
       [157,   0],
       [166,   2],
       [181,   3],
       [191,   2],
       [205,   3],
       [240,   5],
       [247,   2],
       [258,   1],
       [ 12,   2],
       [ 37,   6],
       [ 40,   4],
       [ 42,   5],
       [ 47,   5],
       [ 69,   1],
       [ 71,   0],
       [ 88,   5],
       [ 98,   4],
       [103,   3],
       [111,   1],
       [115,   2],
       [119,   0],
       [120,   5],
       [126,   2],
       [137,   3],
       [140,   1],
       [142,   1],
       [155,   0],
       [162,   4],
       [165,   1],
       [167,   4],
       [173,   5],
       [  1,   0],
       [  2,   3],
       [  5,   5],
       [ 13,   4],
       [ 17,   4],
       [ 19,   5],
       [ 20,   3],
       [ 22,   1],
       [ 27,   0],
       [ 28,   2],
       [ 32,   3],
       [ 34,   6],
       [ 35,

In [16]:
# 原来已经确定好的桶是这些
all_set = {}
for i in range(passed):
    all_set[i] = center_[i][3]

for i in all_set:
    print(i, len(all_set[i]))

0 19
1 34
2 37


In [17]:
# 多出来的用户他们分别对应一些二级类
extra_set

array([[  4,   5],
       [ 54,   0],
       [ 95,   2],
       [105,   6],
       [108,   0],
       [116,   3],
       [129,   6],
       [130,   3],
       [150,   2],
       [157,   0],
       [166,   2],
       [181,   3],
       [191,   2],
       [205,   3],
       [240,   5],
       [247,   2],
       [258,   1],
       [ 12,   2],
       [ 37,   6],
       [ 40,   4],
       [ 42,   5],
       [ 47,   5],
       [ 69,   1],
       [ 71,   0],
       [ 88,   5],
       [ 98,   4],
       [103,   3],
       [111,   1],
       [115,   2],
       [119,   0],
       [120,   5],
       [126,   2],
       [137,   3],
       [140,   1],
       [142,   1],
       [155,   0],
       [162,   4],
       [165,   1],
       [167,   4],
       [173,   5],
       [  1,   0],
       [  2,   3],
       [  5,   5],
       [ 13,   4],
       [ 17,   4],
       [ 19,   5],
       [ 20,   3],
       [ 22,   1],
       [ 27,   0],
       [ 28,   2],
       [ 32,   3],
       [ 34,   6],
       [ 35,

In [18]:
# 把这些类加上已经分好的桶，就是对应的新的类
for i in range(passed, passed + len(dic)):
    temp = []
    for j in range(len(extra_set)):
        if extra_set[j][1] == (i - passed):
            temp.append(extra_set[j][0])
    all_set[i] = temp

for i in all_set:
    print(i, len(all_set[i]))

0 19
1 34
2 37
3 38
4 33
5 38
6 42
7 28
8 36
9 34


# 这个，就是被我分好的全部桶，每个桶都有他对应的集合的标号

In [19]:

all_set

{0: [15,
  29,
  38,
  41,
  44,
  45,
  97,
  101,
  102,
  121,
  132,
  138,
  147,
  160,
  198,
  217,
  279,
  328,
  333],
 1: [3,
  7,
  68,
  70,
  73,
  76,
  100,
  171,
  209,
  273,
  281,
  305,
  327,
  329,
  338,
  21,
  25,
  36,
  39,
  43,
  61,
  64,
  65,
  78,
  81,
  84,
  96,
  104,
  124,
  127,
  131,
  141,
  253,
  267],
 2: [14,
  92,
  99,
  106,
  107,
  110,
  123,
  143,
  153,
  238,
  6,
  8,
  9,
  10,
  11,
  18,
  46,
  50,
  53,
  62,
  63,
  66,
  67,
  74,
  79,
  80,
  85,
  158,
  183,
  189,
  222,
  223,
  262,
  296,
  319,
  331,
  335],
 3: [54,
  108,
  157,
  71,
  119,
  155,
  1,
  27,
  49,
  83,
  87,
  332,
  75,
  145,
  149,
  184,
  235,
  290,
  295,
  51,
  72,
  133,
  192,
  201,
  204,
  211,
  213,
  229,
  239,
  246,
  271,
  282,
  291,
  301,
  307,
  312,
  317,
  330],
 4: [258,
  69,
  111,
  140,
  142,
  165,
  22,
  86,
  93,
  113,
  136,
  146,
  202,
  234,
  274,
  176,
  215,
  244,
  326,
  0,
  23,
  148,

# 这个，就是我准备直接读取的

In [20]:
pk.dump(all_set, open('./pretrain/user_based_10.pk', 'wb'))

In [21]:
for i in range(len(all_set)):
    print(i, len(all_set[i]))

0 19
1 34
2 37
3 38
4 33
5 38
6 42
7 28
8 36
9 34


# 这个就是一行一行将数据放到十个桶里面

In [22]:
tensor = np.array(pk.load(open('datasets/data/WSDREAM/rt.pk', 'rb')))
split_tensor = []
for i in range(args.slices):
    # idx =
    idx = np.array(C[i], dtype = 'int32')
    temp = np.zeros_like(tensor)
    temp[idx] = tensor[idx]
    split_tensor.append(temp)
np.array(split_tensor).shape

(10, 339, 5825)

In [23]:
# processed代表是否使用outlier
!python Experiment.py --dataset rt --interaction NeuCF --rounds 1 --slice_epochs 15 --agg_epochs 10 --density 0.10 --slices 10 --processed 1

2022-12-04 15:14:15  Namespace(agg_epochs=10, agg_type='att', balance=0, batch_size=128, dataset='rt', debug=0, density=0.1, devices='gpu', dimension=64, dropout=0.1, interaction='NeuCF', kmeans_type=1, node2vec=0, node2vec_batchsize=32, node2vec_dim=128, node2vec_epochs=20, node2vec_length=10, node2vec_walk=50, node2vec_windows=3, num_layers=2, order=2, part_iter=50, part_type=1, path='./datasets/data/WSDREAM/', processed=1, retrain=0, rounds=1, save_model=1, slice_epochs=15, slices=10, subagg=1, verbose=0)
2022-12-04 15:14:15  All the experiments have been done!
2022-12-04 15:14:15  Dataset : rt     interaction : NeuCF
2022-12-04 15:14:15  Density : 10.00%, slice_epochs : 15, agg_epochs : 10
2022-12-04 15:14:15  Part_type : 1,    slices : 10,       devices : gpu
2022-12-04 15:14:15  Dataset : rt     interaction : NeuCF
2022-12-04 15:14:15  Density : 10.00%, slice_epochs : 15, agg_epochs : 10
2022-12-04 15:14:15  Part_type : 1,    slices : 10,       devices : gpu


2022-12-04 15:14:15